# Clustering Neighbourhoods in Toronto
## Coursera Capstone Project 

### Harishankar


### Part 1 - Converting Wikipedia html table into a DataFrame

In [1]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
my_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

### Parsing the web html file with BeautifulSoup package

In [3]:
r=requests.get(my_url)
page=bsoup(r.text,"html.parser")
rtable=page.table

### 'The htlm 'page'  indicates that there is a 'table class' - let's find that table 

In [6]:
results=rtable.find_all('tr')
nrows=len(results)
nrows

290

In [8]:
header=results[0].text.split()
header

['Postcode', 'Borough', 'Neighbourhood']

### Let's check some  rows in order to prepare to build the loop that will extract all cells into a DataFrame.  For example, let's examine row 85


In [9]:
results[85].text

'\nM5J\nDowntown Toronto\nToronto Islands\n'

In [10]:
results[85].text.split('\n')

['', 'M5J', 'Downtown Toronto', 'Toronto Islands', '']

In [11]:
results[85].text.split('\n')

['', 'M5J', 'Downtown Toronto', 'Toronto Islands', '']

In [12]:
Postcode=results[85].text.split('\n')[1]
Postcode

'M5J'

In [13]:
Borough=results[85].text.split('\n')[2]
Borough

'Downtown Toronto'

In [14]:
Neighborhood=results[85].text.split('\n')[3]
Neighborhood

'Toronto Islands'

## Iteration loop to extract all cells into a dataframe df

In [15]:
# iteration loop to harvest all records

records =[]
n=1
while n < nrows :
    Postcode=results[n].text.split('\n')[1]
    Borough=results[n].text.split('\n')[2]
    Neighborhood=results[n].text.split('\n')[3]
    records.append((Postcode, Borough,Neighborhood))
    n=n+1

df=pd.DataFrame(records, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.head(5)



,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [18]:
# How many rows have Borough equal to 'Not assigned'?
df[df['Borough']=='Not assigned'].count()

PostalCode       77
Borough          77
Neighbourhood    77
dtype: int64

In [19]:
# drops those rows where 'Not assigned' appears in column '[Borough]'
df1=df[~df.Borough.str.contains("Not assigned")]
df1=df1.reset_index(drop=True)

In [20]:
df1.head(5)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [23]:
df1.loc[df1['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df1['Borough']

In [24]:
df1.head()           # Now, there are no rows with 'Not assigned' strings anymore

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [26]:
postalcodes = df1['PostalCode'].nunique()
boroughs = df1['Borough'].nunique()
neighbourhoods= df1['Neighbourhood'].nunique()
print('Unique Postalcodes : ' + str(postalcodes))
print('Unique Boroughs  : '+ str(boroughs))
print('Unique Neighbourhoods  :' + str(neighbourhoods))

     

Unique Postalcodes : 103
Unique Boroughs  : 11
Unique Neighbourhoods  :210


### There are 103 Unique Postcodes and 11 Unique Borough with 210 Neighbourhoods

In [27]:
#df1.groupby(['PostalCode','Borough','Neighbourhood']).size().reset_index(name='Count').head()

## Let's Consolidate the dataframe to each unique PostalCodes and aggregated Neighbourhoods

In [28]:
# Starting DataFrame df1
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [29]:
nrows1=len(df1)
nrows1

212

In [30]:
# DataFrame df2 also has 212 rows or cells
df2=df1
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [31]:
# df2.index

In [32]:
# I will use two pointers, 'n and m' to sweep through the dataframe. So 'n' will vary from
#  0 to 211, and 'm ' will vary from 1 to 212
nrows2=len(df2)-1
nrows2

211

## Loop Algorithm to extract the consolidate dataframe df2 with unique postalcodes.
### We expect a df2 with 103 rows because that is the number of unique postalcodes

In [33]:
n=0

while n < nrows2 :
    post1=df2.iloc[n,0]
    #post1
    m=n+1
    post2=df2.iloc[m,0]
    #post2
    neigh1=df2.iloc[n,2]
    neigh2=df2.iloc[m,2]
    if post1==post2:
        df2.Neighbourhood[n,2] = neigh1=neigh1+','+neigh2
        #df2 = df2[df2.Neighbourhood != 'neigh2']
        df2=df2.drop(df2.index[m])
        nrows2=nrows2-1
        df2 = df2.reset_index(drop=True)
    else:
        n=n+1


#df2 = df2.reset_index(drop=True)
# print(post1, post2, 'Nigh1 is...'+ neigh1,',,,,,,,','Nigh2 is...' + neigh2, n,m)
df2.index


RangeIndex(start=0, stop=103, step=1)

In [34]:
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw..."
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [35]:
df2.index

RangeIndex(start=0, stop=103, step=1)

In [36]:
#df2.sort_values('PostalCode').head(15)
df2.head(15)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw..."
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"
